In [ ]:
"""
Interactive NHL Data Explorer with ipywidgets
Must be run in a Jupyter notebook
"""

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

class InteractiveNHLExplorer:
    """Simple interactive tool to explore NHL data"""
    
    def __init__(self):
        # Load cleaned (tidy) data with real player names
        tidy_data_path = Path(r"C:\Users\AgeTeQ\Desktop\data\classes\DS\tp1\project-template\data\tidy")
        self.data = self._load_tidy_data(tidy_data_path)
        self.current_game_data = None
        
    def _load_tidy_data(self, tidy_dir):
        """Load all CSV files from the tidy folder"""
        if not tidy_dir.exists():
            return pd.DataFrame()
        
        try:
            # Find all CSV files in folder
            csv_files = list(tidy_dir.rglob("*.csv"))
            
            all_data = []
            for csv_file in csv_files:
                try:
                    df = pd.read_csv(csv_file)
                    
                    # Add distance from net if missing
                    if 'distance_from_net' not in df.columns:
                        df = self._calculate_distance_for_data(df)
                    
                    all_data.append(df)
                except Exception:
                    continue
            
            # Combine all CSV files into one DataFrame
            if all_data:
                return pd.concat(all_data, ignore_index=True)
            else:
                return pd.DataFrame()
                
        except Exception:
            return pd.DataFrame()
    
    def _calculate_distance_for_data(self, df):
        """Compute distance from the net for each event"""
        df['x_coord'] = pd.to_numeric(df['x_coord'], errors='coerce')
        df['y_coord'] = pd.to_numeric(df['y_coord'], errors='coerce')
        
        def calculate_distance(row):
            # If no coordinates, skip
            if pd.isna(row['x_coord']) or pd.isna(row['y_coord']):
                return None
            # Left or right net
            net_x = -89 if row['x_coord'] < 0 else 89
            # Euclidean distance
            return ((row['x_coord'] - net_x) ** 2 + row['y_coord'] ** 2) ** 0.5
        
        valid_coords = df['x_coord'].notna() & df['y_coord'].notna()
        df.loc[valid_coords, 'distance_from_net'] = df[valid_coords].apply(calculate_distance, axis=1)
        df['is_goal'] = (df['event_type'] == 'GOAL').astype(int)
        
        return df
    
    def create_interactive_widget(self):
        """Create the dropdowns, slider, and plots"""
        if self.data.empty:
            return
        
        # Dropdowns for season and game type
        seasons = sorted(self.data['season'].unique()) if 'season' in self.data.columns else ['All Seasons']
        game_types = sorted(self.data['game_type'].unique()) if 'game_type' in self.data.columns else ['General']
        
        self.season_dropdown = widgets.Dropdown(
            options=seasons,
            value=seasons[0],
            description='Season:'
        )
        
        self.game_type_dropdown = widgets.Dropdown(
            options=game_types,
            value=game_types[0],
            description='Game Type:'
        )
        
        # Dropdown for specific game
        self.game_dropdown = widgets.Dropdown(description='Game ID:')
        
        # Slider to move between events
        self.event_slider = widgets.IntSlider(
            value=0,
            min=0,
            max=100,
            step=1,
            description='Event Index:',
            continuous_update=False
        )
        
        self.output = widgets.Output()
        
        # Update content when dropdowns or slider change
        def on_season_change(change):
            self._update_game_list()
            
        def on_game_type_change(change):
            self._update_game_list()
            
        def on_game_change(change):
            self._update_event_slider()
            self._plot_current_event()
            
        def on_event_change(change):
            self._plot_current_event()
        
        # Link events to functions
        self.season_dropdown.observe(on_season_change, names='value')
        self.game_type_dropdown.observe(on_game_type_change, names='value')
        self.game_dropdown.observe(on_game_change, names='value')
        self.event_slider.observe(on_event_change, names='value')
        
        # Display all widgets together
        display(widgets.VBox([
            widgets.HBox([self.season_dropdown, self.game_type_dropdown]),
            self.game_dropdown,
            self.event_slider,
            self.output
        ]))
        
        # Initialize game list
        self._update_game_list()
    
    def _update_game_list(self):
        """Filter available games based on selected season and type"""
        season = self.season_dropdown.value
        game_type = self.game_type_dropdown.value
        
        games = self.data[
            (self.data['season'] == season) & 
            (self.data['game_type'] == game_type)
        ]['game_id'].unique()
        
        self.game_dropdown.options = sorted(games)
        if len(games) > 0:
            self.game_dropdown.value = games[0]
    
    def _update_event_slider(self):
        """Set slider limits based on number of events in the game"""
        game_id = self.game_dropdown.value
        if game_id:
            game_data = self.data[self.data['game_id'] == game_id]
            self.current_game_data = game_data.reset_index(drop=True)
            self.event_slider.max = len(game_data) - 1 if len(game_data) > 0 else 0
            self.event_slider.value = 0
    
    def _plot_current_event(self):
        """Show the rink, all events, and event details"""
        with self.output:
            clear_output(wait=True)
            
            if self.current_game_data is None or len(self.current_game_data) == 0:
                return
            
            event_index = self.event_slider.value
            if event_index >= len(self.current_game_data):
                event_index = len(self.current_game_data) - 1
                self.event_slider.value = event_index
            
            event = self.current_game_data.iloc[event_index]
            
            # Make 3 side-by-side plots
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
            
            # Left: current event
            self._draw_ice_rink(ax1)
            self._plot_single_event(ax1, event, event_index)
            ax1.set_title('Current Event Location')
            
            # Middle: all events (shots + goals)
            self._draw_ice_rink(ax2)
            self._plot_all_events(ax2)
            shots_count = len(self.current_game_data[self.current_game_data['is_goal'] == 0])
            goals_count = len(self.current_game_data[self.current_game_data['is_goal'] == 1])
            ax2.set_title(f'All Events\nShots: {shots_count}, Goals: {goals_count}')
            
            # Right: text info about current event
            self._display_event_info(ax3, event, event_index)
            ax3.set_title('Event Info')
            
            plt.tight_layout()
            plt.show()
    
    def _draw_ice_rink(self, ax):
        """Draw rink image or a simple version if image missing"""
        rink_path = r"C:\Users\AgeTeQ\Desktop\data\classes\DS\tp1\project-template\figures\nhl_rink.png"
        
        try:
            rink_img = plt.imread(rink_path)
            ax.imshow(rink_img, extent=[-100, 100, -42.5, 42.5])
        except:
            # Draw a simple version if no image
            rink = patches.Rectangle((-100, -42.5), 200, 85, linewidth=2, 
                                   edgecolor='black', facecolor='lightblue', alpha=0.3)
            ax.add_patch(rink)
            ax.axvline(x=-25, color='blue', linewidth=2)
            ax.axvline(x=25, color='blue', linewidth=2)
            ax.axvline(x=-89, color='red', linewidth=2)
            ax.axvline(x=89, color='red', linewidth=2)
            ax.axvline(x=0, color='red', linestyle='--', linewidth=1)
        
        ax.set_xlim(-100, 100)
        ax.set_ylim(-42.5, 42.5)
        ax.set_aspect('equal')
    
    def _plot_single_event(self, ax, event, event_index):
        """Show one specific event on the rink"""
        if pd.notna(event.get('x_coord')) and pd.notna(event.get('y_coord')):
            x, y = event['x_coord'], event['y_coord']
            
            # Color and marker change if goal or shot
            if event.get('is_goal') == 1:
                color, marker, size, label = 'red', '*', 200, 'Goal'
            else:
                color, marker, size, label = 'blue', 'o', 120, 'Shot'
            
            ax.scatter(x, y, c=color, marker=marker, s=size, alpha=0.9,
                       edgecolors='white', linewidth=2, label=label)
            
            # Add label with event number
            ax.annotate(f"Event {event_index + 1}", xy=(x, y), xytext=(x + 8, y + 8),
                       fontsize=9, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.8))
            ax.legend(loc='upper right', fontsize=8)
        else:
            ax.text(0, 0, "No coordinates", ha='center', va='center',
                   fontsize=12, bbox=dict(boxstyle="round,pad=0.5", facecolor="yellow", alpha=0.8))
    
    def _plot_all_events(self, ax):
        """Show all shots and goals for the selected game"""
        if self.current_game_data is None:
            return
            
        shots = self.current_game_data[self.current_game_data['is_goal'] == 0]
        goals = self.current_game_data[self.current_game_data['is_goal'] == 1]
        
        if not shots.empty:
            ax.scatter(shots['x_coord'], shots['y_coord'], c='blue', alpha=0.6, s=50, label='Shots')
        if not goals.empty:
            ax.scatter(goals['x_coord'], goals['y_coord'], c='red', alpha=1.0, s=100, marker='*', label='Goals')
        
        ax.legend(loc='upper right')
    
    def _display_event_info(self, ax, event, event_index):
        """Show event info like player, team, and coordinates"""
        ax.axis('off')
        
        if self.current_game_data is not None:
            player_name = event.get('player_name', 'N/A')
            goalie_name = event.get('goalie_name', 'N/A')
            
            info_text = f"""
EVENT {event_index + 1} of {len(self.current_game_data)}

Game ID: {event.get('game_id', 'N/A')}
Season: {event.get('season', 'N/A')}
Game Type: {event.get('game_type', 'N/A')}

Event Type: {event.get('event_type', 'N/A')}
Period: {event.get('period', 'N/A')}
Time: {event.get('period_time', 'N/A')}
Team: {event.get('team_name', 'N/A')}
Player: {player_name}
Goalie: {goalie_name}
Shot Type: {event.get('shot_type', 'N/A')}

X: {event.get('x_coord', 'N/A')}
Y: {event.get('y_coord', 'N/A')}
Distance: {event.get('distance_from_net', 'N/A')} ft
Goal: {'Yes' if event.get('is_goal') == 1 else 'No'}
"""
            ax.text(0.05, 0.95, info_text, transform=ax.transAxes, fontsize=9, 
                    verticalalignment='top', fontfamily='monospace', linespacing=1.5,
                    bbox=dict(boxstyle="round,pad=1", facecolor="lightgray", alpha=0.8))

def launch_interactive_tool():
    """Start the NHL explorer"""
    explorer = InteractiveNHLExplorer()
    explorer.create_interactive_widget()
    return explorer

# Run inside Jupyter
if __name__ == "__main__":
    plt.rcParams['figure.figsize'] = [20, 6]
    plt.rcParams['font.size'] = 10
    
    explorer = launch_interactive_tool()
